In [ ]:
import pandas as pd
import numpy as np
import statistics as sts
import tensorflow as tf
import csv

In [ ]:
df=pd.read_csv("../input/drug-classification/drug200.csv")
pd.set_option('display.max_columns', 500)
print(df[df.columns].isnull().sum())
df

In [ ]:
def z_score(arr):
    ret=[]
    arr = list(arr)
    arr.sort()
    mean = sts.mean(arr)
    std = sts.stdev(arr)
    for i in arr:
        ret.append(abs((i-mean)/std))
    return ret

cont=['Age','Na_to_K']
for i in cont:
    df[('z'+i)]=z_score(df[i])

df

In [ ]:

df=df.drop(df[(df['zAge']>3) | (df['zNa_to_K']>3)].index)

df=df.drop(columns=['zAge','zNa_to_K'])



In [ ]:
df['Age']=df["Age"]/df["Age"].max()
df['Na_to_K']=df['Na_to_K']/df['Na_to_K'].max()

df

In [ ]:
df=pd.get_dummies(df)
df

In [ ]:
df=df.drop(columns=['Sex_M','BP_NORMAL','Cholesterol_NORMAL'])

In [ ]:
df

# Sex_F : 1 = Female , 0 = Male
# bp_high,bp_low : (1,0) is high , (0,0) is normal bp , (0,1) is low bp
# chloestrol_high : 1 is high and 0 is low
#class of drug_y=0 and drug_a=1,drug_b=2,drug_c=3,drug_x=4

In [ ]:
x=df.iloc[:,:6].values
y=df.iloc[:,6:].values

In [ ]:
tf.random.set_seed(42)

In [ ]:

model=tf.keras.models.Sequential([
    tf.keras.layers.Dense(100,activation="relu",input_dim=6),
    tf.keras.layers.Dense(110,activation="relu"),
    tf.keras.layers.Dense(100,activation="relu"),
    tf.keras.layers.Dense(5,activation="softmax")

])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.002999),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=["accuracy"]
              )
model.fit(xtrain,ytrain,epochs=49)

In [ ]:
model.evaluate(xtest,ytest)

In [ ]:
#Evaluating Generalization Capablity of the model
ypred=np.round(model.predict(xtest)).astype(int)
ypred==ytest
#Model has Predicted all the test set data (unseen data) results correctly 

In [ ]:
y_res_pred=[]
y_res_test=[]
for i in ypred:
    y_res_pred.append(np.argmax(i))
for j in ytest:
    y_res_test.append(np.argmax(j))


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_res_test,y_res_pred)*100
#100% accuracy